- 미니 배치 학습
: 데이터가 많은 경우 일부를 추려 근사치로 이용하여 학습을 수행한다. 

In [1]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

In [2]:
(x_train, t_train), (x_test, t_test) =\
load_mnist(normalize=True, one_hot_label=True)

print("x_train shape:::",x_train.shape)
print("t_train shape:::",t_train.shape)

x_train shape::: (60000, 784)
t_train shape::: (60000, 10)


**!!!**  \ -> 무엇인가.. 자동생성해주네

one_hot_label : 정답 위치의 원소만 1 나머지는 0

    - dataset에서 무작위로 10장만 뺀다

In [4]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)

x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

print("x_batch:::", x_batch)
print("t_batch:::", t_batch)
print("batch_mask:::", batch_mask)

x_batch::: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
t_batch::: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
batch_mask::: [20307 32888 21923 12699 19662 12515 27174 32649 15408 26864]


뭐여...뭐야??
아 batch_mask가 데이터의 인덱스가 됨. 인덱스마다 배열이 들어있음
np.random.choice : 지정한 범위의 수중에서 무작위로 원하는 개수만 꺼낼 수 있다.

- 배치용 교차 엔트로피 오차 구현
미니 배치 같은 데이터를 처리할 수 있는 교차 엔트로피 구현

In [5]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)        
    batch_size = y.shape[0]
    
    return -np.sum(t * np.log(y)) / batch_size

y는 신경망의 출력, t는 정답 레이블
y가 1차원이라면, *즉 데이터 하나당 교차 엔트로피 오차를 구하는 경우* reshape 함수로 데이터의 형상을 바꿔준다. 그리고 배차(?)의 크기로 나눠 정규화하고 *이미지 1장당 편귱 교차 엔트로피 오차를 계산*

- 정답 레이블이 원핫코딩이 아니라 숫자 레이블로 주어졌을 경우 구현

In [3]:
def cross_entropy_error2(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    
    return -np.sum(np.lag(y[np.arrange(batch_size), t])) / batch_size

위 구현의 핵심 : 원-핫 인코딩일 때 t가 0인 원소는 교차 엔트로피 오차도 0이므로 그 계산은 무시해도 좋다는 것. 즉 정답에 해당하는 신경망의 출력만으로 교차 엔트로피 오차를 계산할 수 있다. 그래서 np.log(y) 부분을 np.lag(y[np.arrange(batch_size), t]로 구현

np.arrange(batch_size)는 0부터 batch_size-1 까지 배열을 생성한다. 즉 batch_size가 5이면 [0, 1, 2, 3, 4] 배열을 생성한다. t에는 레이블이 [2, 7, 0, 9, 4]와 같이 저장되 있으므로 y[np.arrange(batch_size), t]는 각 데이터의 정답 레이블에 해당하는 신경망의 출력을 추출한다. 

위 예시로 보면 [y[0, 2], y[1, 7], y[2, 0], y[3, 9], y[4, 4]]를 반환한다.

- 수치미분

경사법에서는 기울기 값을 기준으로 나아갈 방향을 정한다. 

1. 미분
어떤 작은 변화가 함수를 얼마나 변화시키는가

**나쁜 구현 예**

In [4]:
def numerical_diff(f, x):
    h = 10e-50
    return (f(x + h) - f(x)) / h

h에 가능한한 무한히 0에 가깝게 하기 위해서 10e-50을 사용했다. 그러나 이 방식은 **반올림 오차** 문제를 일으킨다.
반올림 오차는 작은값이 생략되어 최종 계산 결과에 오차가 생기게 한다. 즉 0.0이 되버려서 올바른 표현이 어렵다

또, 진정한 미분은 x 위치의 함수의 기울기에 해당하지만 위 구현에서의 미분은 (x+h)와 x 사이의 기울기에 해당한다. 그래서 진정한 미분과 일치 하지 않음. 이 차이는 h를 무한히 0으로 좁히는 것이 불가능해서 생기는 문제로 수치 미분에는 오차가 포함됨.
이 오차를 줄이기 위해 (x + h)와 (x - h)일 때의 함수f의 차분을 계산하는 방법을 쓰기도 한다.